负责把vnpy格式的1min csv合成为vnpy格式的日线 csv

In [1]:
import pandas as pd
import glob
import os
import time

In [2]:
data_folder_path = r"D:\data\vnpy\future\1_min"
data_path_list = glob.glob(os.path.join(data_folder_path, "*.csv"))
data_path_list = [path for path in data_path_list if "#"not in path and "MGC" == os.path.basename(path)[0:3]]

In [4]:
day_bar_data_folder = r"D:\data\vnpy\future\day"

In [5]:
len(data_path_list)

61

In [6]:
for number, path in enumerate(data_path_list):
    start_time = time.time()
    print("processing file {}/{}".format(number, len(data_path_list)))
    # read one min bar data
    frame = pd.read_csv(path)
    datetime_format = '%Y%m%d %H:%M:%S'
    frame.loc[:, 'datetime'] = pd.to_datetime(frame['datetime'], format=datetime_format)
    frame.loc[:, 'date'] = frame.loc[:, 'datetime'].apply(lambda x: x.date())
    float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
    for col in float_columns:
        frame.loc[:, col] = frame[col].astype('float')
    frame = frame.sort_values(by="datetime")
    # build day bar data
    day_bar_frame = pd.DataFrame(columns=['open',
                                          'high',
                                          'low',
                                          'close',
                                          'volume',
                                          'date',
                                          'exchange',
                                          'open_interest',
                                          'interval',
                                          'symbol' ])
    for x, sub_data in frame.groupby("date"):
        new_row = pd.DataFrame(columns=['open',
                                        'high',
                                        'low',
                                        'close',
                                        'volume',
                                        'date',
                                        'exchange',
                                        'open_interest',
                                        'interval',
                                        'symbol' ])
        new_row.loc[0,"open"] = sub_data.iloc[0,:].loc["open"]
        new_row.loc[0,"high"] = sub_data.loc[:, "high"].max()
        new_row.loc[0,"low"] = sub_data.loc[:, "low"].min()
        new_row.loc[0,"close"] = sub_data.iloc[len(sub_data)-1,:].loc["close"]
        new_row.loc[0,"volume"] = sub_data.loc[:, "volume"].sum()
        new_row.loc[0,"date"] = x
        new_row.loc[0,"exchange"] = sub_data.iloc[0,:].loc["exchange"]
        new_row.loc[0,"open_interest"] = 0.0
        new_row.loc[0,"interval"] = sub_data.iloc[0,:].loc["interval"]
        new_row.loc[0,"symbol"] = sub_data.iloc[0,:].loc["symbol"]
        day_bar_frame = pd.concat([day_bar_frame, new_row])

    base_name = os.path.basename(path)
    day_bar_path = os.path.join(day_bar_data_folder, base_name)
    day_bar_frame.to_csv(day_bar_path, index=False)
    end_time = time.time()
    print("time used: {} sec".format((end_time - start_time)))

processing file 0/61
time used: 1.240405559539795 sec
processing file 1/61
time used: 2.4688515663146973 sec
processing file 2/61
time used: 2.7459685802459717 sec
processing file 3/61
time used: 2.249922513961792 sec
processing file 4/61
time used: 2.1279802322387695 sec
processing file 5/61
time used: 2.6267685890197754 sec
processing file 6/61
time used: 3.008558511734009 sec
processing file 7/61
time used: 2.591758966445923 sec
processing file 8/61
time used: 3.423511266708374 sec
processing file 9/61
time used: 3.7901883125305176 sec
processing file 10/61
time used: 1.6543910503387451 sec
processing file 11/61
time used: 2.4478256702423096 sec
processing file 12/61
time used: 3.162520408630371 sec
processing file 13/61
time used: 2.695706367492676 sec
processing file 14/61
time used: 2.1649632453918457 sec
processing file 15/61
time used: 2.7796716690063477 sec
processing file 16/61
time used: 3.226968288421631 sec
processing file 17/61
time used: 3.1047258377075195 sec
processing